<a href="https://colab.research.google.com/github/Amit-712/Breast-Cancer-Detection/blob/main/BreastCancerInterface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.7 MB/s eta 0:00:00


In [ ]:
from google.colab import files

uploaded = files.upload()


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

import torch
import torch.nn as nn
import torchvision.models as models

model_name = 'vgg16'
#Customvgg16 class inherit from nn.Module
class CustomVGG16(nn.Module):
    def __init__(self, num_classes=2):
        super(CustomVGG16, self).__init__()

        # Loading pre-trained VGG16 model from tourchvision.models
        vgg16 = models.vgg16(pretrained=True)

        # Extracting the features and avgpool layers frm pretrainedmodel
        self.features = vgg16.features
        self.avgpool = vgg16.avgpool

        # Define a new classifier nm.Sequential
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096), #input and output
            nn.ReLU(inplace=True), #for activation
            nn.Dropout(), #for regularization
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):

        x = self.features(x) # Passing the input through the features layer
        x = self.avgpool(x) # Using the avgpool layer
        x = torch.flatten(x, 1) # Reshaping the output to a 2D tensor
        x = self.classifier(x) # Passing the reshaped output to the custom classifier
        return x

In [ ]:
vgg16_path = '/content/checkpoint.pt'

vgg16_model = CustomVGG16(num_classes=2)
vgg16_model.load_state_dict(torch.load(vgg16_path, map_location = torch.device('cpu'), weights_only=False))
vgg16_model.eval()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 80.9MB/s]


FileNotFoundError: [Errno 2] No such file or directory: '/content/checkpoint.pt'

In [ ]:
with open(vgg16_path, "rb") as f:
    print(f.read(10))


b'8{\x97\xa1\xbb\xe1\xf7\xd2\xbbJ'


In [ ]:
import gradio as gr
import torch
from PIL import Image

def predict(image):
    # preprocess image and get model output
    result = model(image)  # dummy placeholder
    return "Benign" if result < 0.5 else "Malignant"

gr.Interface(fn=predict,
             inputs=gr.Image(type="pil"),
             outputs="text",
             title="Breast Cancer Detector").launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://74b7fd3a7d793dd90a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
from PIL import Image
import torch
import torchvision.transforms as transforms

# Load your trained model
model = torch.load('checkpoint.pt', map_location=torch.device('cpu'))
model.eval()

# Define preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Update if RGB or different stats
])

# Prediction function
def predict(image):
    image = transform(image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        output = model(image)
        probs = torch.softmax(output, dim=1)
        class_idx = torch.argmax(probs).item()
        confidence = probs[0][class_idx].item()

    label_map = {0: "Benign", 1: "Malignant"}
    return {
        "Prediction": label_map[class_idx],
        "Confidence": f"{confidence*100:.2f}%"
    }

# Gradio UI
interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil", label="Upload X-ray or Mammogram"),
    outputs=[
        gr.Label(label="Prediction"),
        gr.Textbox(label="Confidence Score")
    ],
    title="Breast Cancer Detection",
    description="Upload a mammogram to detect if the tumor is benign or malignant using a CNN model.",
    theme="default",
    live=False,
)

interface.launch()


FileNotFoundError: [Errno 2] No such file or directory: 'checkpoint.pt'